In [40]:
import json
from unskript import nbparams
from unskript.fwk.workflow import Task, Workflow
from unskript.secrets import ENV_MODE, ENV_MODE_LOCAL

env = {"ENV_MODE": "ENV_MODE_LOCAL"}
secret_store_cfg = {"SECRET_STORE_TYPE": "SECRET_STORE_TYPE_LOCAL"}

paramDict = {"CloudTrailArn": "arn:aws:cloudtrail:us-west-2:100498623390:trail/management-events", "policy_name": "generated_iam_policy_11", "reference_iam_arn": "arn:aws:iam::100498623390:user/doug-billing-s3", "region": "us-west-2", "threshold": float(24), "user_name": "Doug_generated_iam_14"}
paramsJson = json.dumps(paramDict)
nbParamsObj = nbparams.NBParams(paramsJson)
CloudTrailArn = nbParamsObj.get('CloudTrailArn')
policy_name = nbParamsObj.get('policy_name')
reference_iam_arn = nbParamsObj.get('reference_iam_arn')
region = nbParamsObj.get('region')
threshold = nbParamsObj.get('threshold')
user_name = nbParamsObj.get('user_name')
w = Workflow(env, secret_store_cfg, None, global_vars=globals())

<h1>Create IAM Access Policy based on Usage</h1>
<p>&nbsp;</p>
<p>IAM Access should follow the policy of least privilege. This means that the credentials give "exactly enough" access to &nbsp;perform the requried task, but no more.&nbsp; That way, if the credentials were ever to be compromised, the blast radius is minimized.</p>
<p>This RunBook will take an active IAM profile, and analyze it's access over the last &lt;threshold&gt; hours.&nbsp; Using CloudTrail logs, we can determine what was accessed, and create a new IAM profile that gives access to just these features,</p>
<p>&nbsp;</p>
<h2>Input parameters:</h2>
<h3>&nbsp;Credentials</h3>
<p>You will need two IAM accounts to complete this Runbook:</p>
<ol>
<li>admin_iam:&nbsp; Use these credentials to run each Action - creating IAM policies requires admin access.</li>
<li>reference_iam_arn: This parameter should have the ARN for the reference IAM account.&nbsp; We'll use the activity from this account to generate a new IAM policy.</li>
</ol>
<h3>Inputs</h3>
<ol>
<li>CloudTrail ARN:&nbsp; This is the ARN of the CloudTrail log that you wish to query. IF you are not sure which ARN you wish to use, you can use the "AWS Describe Cloudtrails" Action to get a list of all your trails.</li>
<li>Region: the AWS Region.</li>
<li>threshold: The number of hours of cloudtrail logs to exaine for activity,</li>
<li>policy_name: the name of your new IAM access policy</li>
<li>user_name: The new IAM user you will create with the policy_name attached.</li>
</ol>
<h2>Steps</h2>
<ol>
<li><strong>AWS Describe Cloudtrails:</strong> Gets a list of all the Cloudtrail logs in a region. Use this to get your CloudTrailARN.&nbsp; This step requires a region.
<ol>
<li>If you know the Cloudtrail ARN - you can safely delete this Step</li>
</ol>
</li>
<li><strong>AWS Start IAM Policy Generation</strong>: Begins the process of creating a IAM Policy.&nbsp; Note that you can only create one policy at a time, so if a previous policy is still in progress, this may throw an error.
<ol>
<li>Inputs:&nbsp;
<ol>
<li>Region: AWS Region</li>
<li>CloudTrailARN: ARN of the Cloudtrail - default is to use the runbook input parameter.</li>
<li>IAMPrincipalARN.&nbsp; The IAM user whose access is being duplicated</li>
<li>AccessRole: IAM access role for "AccessAnalyzerMonitorServiceRole". You'll need to create this in your AWS Console</li>
<li>hours: number of hours of logs to examine:&nbsp; Default is threshold input parameter.</li>
</ol>
</li>
<li>Output:
<ol>
<li>JobId - the UUID of the Policy creation</li>
</ol>
</li>
</ol>
</li>
<li><strong>AWS Get Generated Policy</strong> Gets the policy generated in the previous step.&nbsp; Note that generation can take some time, and the response (in variable generatedPolicy) has a status (generatedPolicy['jobDetails']['status']). When this reads "SUCCEEDED", the runbook can be contiued.)
<ol>
<li>Inputs
<ol>
<li>Region</li>
<li>JobId (from the start generation step)</li>
</ol>
</li>
<li>Output:
<ol>
<li>Response from the Call</li>
</ol>
</li>
</ol>
</li>
<li><strong>AWS Get Account Number</strong> This Action retrieves your AWS Account number.&nbsp; It is required to clean up the policy that is returned from step 3.&nbsp; Output is the accountNumber</li>
<li><strong>Clean Up Policy: </strong>This step reads the generated policy (generatedPolicy['generatedPolicyResult']['generatedPolicies'][0]['policy']), and does some cleanup.&nbsp; In the generated policy, there are variables in the policy that must be given concrete values.&nbsp; In our test runs, the following changes have been made (you may need to do further cleanup in this Action to continue:
<ol>
<li>policy = policy.replace('${Region}', "us-west-2")</li>
<li>policy = policy.replace('${Account}', accountNumber)</li>
</ol>
</li>
<li><strong>AWS Create IAM Policy</strong> Takes the policy that was cleaned in the last step, and creates a new policy in your AWS Account. The name of the policy is based on the policy_name input variable,&nbsp; There can only be one unique value, so once a policy is created, this will need to be changed.</li>
<li><strong>Create New IAM User</strong> Creates a new IAM user (using the user_name input).&nbsp;&nbsp;</li>
<li><strong>AWS Attach New Policy to User&nbsp;</strong> Attaches the created policy to the created user</li>
</ol>
<p>&nbsp;</p>

In [5]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field, SecretStr
from typing import Dict, List
import pprint

from beartype import beartype
@beartype
def aws_describe_cloudtrail_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_describe_cloudtrail(handle, region:str) -> Dict:
    # Create a client object for CloudTrail
    cloudtrail_client = handle.client('cloudtrail', region_name=region)

    # Use the describe_trails method to get information about the available trails
    trails = cloudtrail_client.describe_trails()


    return trails

task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "doug_admin",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "ccdef705-2e5a-4bc5-9709-fbb8eb1057df"
}''')
task.configure(inputParamsJson='''{
    "region": "region"
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_describe_cloudtrail, lego_printer=aws_describe_cloudtrail_printer, hdl=hdl, args=args)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '1057',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Thu, 13 Apr 2023 21:50:22 GMT',
                                      'x-amzn-requestid': 'c1e5cd2a-2e9c-448d-9ac1-30a34ef24ebe'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'c1e5cd2a-2e9c-448d-9ac1-30a34ef24ebe',
                      'RetryAttempts': 0},
 'trailList': [{'HasCustomEventSelectors': True,
                'HasInsightSelectors': False,
                'HomeRegion': 'us-west-2',
                'IncludeGlobalServiceEvents': True,
                'IsMultiRegionTrail': True,
                'IsOrganizationTrail': False,
                'LogFileValidationEnabled': False,
                'Name': 'management-events',
                'S3BucketName': 'aws-cloudtrail-logs-100498623390-f026afa6',
                'TrailARN': 'arn:aws:cloudtrail:us-west-2:

In [6]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field, SecretStr
from typing import Dict, List
import pprint

from beartype import beartype
@beartype
def aws_list_all_iam_users_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_list_all_iam_users(handle) -> List:
    """aws_list_all_iam_users lists all the IAM users

        :type handle: object
        :param handle: Object returned from Task Validate

        :rtype: Result List of all IAM users
    """
    client = handle.client('iam')
    users_list=[]
    response = client.list_users()
    try:
        for x in response['Users']:
            users_list.append(x['UserName'])
    except Exception as e:
        users_list.append(e)
    return users_list


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "doug_admin",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "ccdef705-2e5a-4bc5-9709-fbb8eb1057df"
}''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_all_iam_users, lego_printer=aws_list_all_iam_users_printer, hdl=hdl, args=args)

['123123123',
 'abhishek',
 'amit',
 'cfuser',
 'DODSLC',
 'doug-billing-s3',
 'doug_admin',
 'doug_generated_iam1',
 'Doug_generated_iam_5',
 'Doug_generated_iam_7',
 'Doug_generated_iam_8',
 'Doug_generated_iam_900',
 'doug_local_Docker',
 'doug_test',
 'doug_test_iam',
 'ir-bot',
 'jayasimha',
 'Jenkins',
 'jupyterlab-user',
 'mir',
 's3-user',
 'secretmanager-user',
 'ses-user',
 'shloka',
 'shloka-docker',
 'singleAMIUser-dev',
 'Test',
 'test123',
 'testing_environment',
 'TestRunbook',
 'yura_test']


In [44]:
##
# Copyright (c) 2023 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field, SecretStr
from typing import Dict, List
import pprint
from datetime import datetime, timedelta

from beartype import beartype
@beartype
def AWS_Start_IAM_Policy_Generation_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def AWS_Start_IAM_Policy_Generation(handle, region:str, CloudTrailARN:str, IAMPrincipalARN:str, AccessRole:str, hours:float) -> str:

    client = handle.client('accessanalyzer', region_name=region)
    policyGenerationDict = {'principalArn': IAMPrincipalARN}
    myTrail = {'cloudTrailArn': CloudTrailARN,
                   'regions': [region],
                   'allRegions': False
              }
    endTime = datetime.now()
    endTime = endTime.strftime("%Y-%m-%dT%H:%M:%S")
    startTime = datetime.now()- timedelta(hours =hours)
    startTime =startTime.strftime("%Y-%m-%dT%H:%M:%S")
    response = client.start_policy_generation(    
        policyGenerationDetails=policyGenerationDict,
        cloudTrailDetails={
            'trails': [myTrail],
            'accessRole': AccessRole,
            'startTime': startTime,
            'endTime': endTime
        }
    )
    jobId = response['jobId']
    return jobId

task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "aws_doug_admin",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "48be6a4f-65a0-4fb1-bad1-2db778429f87"
}''')
task.configure(inputParamsJson='''{
    "AccessRole": "\\"arn:aws:iam::100498623390:role/service-role/AccessAnalyzerMonitorServiceRole_CTBKDXMCCK\\"",
    "CloudTrailARN": "CloudTrailArn",
    "IAMPrincipalARN": "reference_iam_arn",
    "hours": "float(24)",
    "region": "region"
    }''')
task.configure(outputName="jobId")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(AWS_Start_IAM_Policy_Generation, lego_printer=AWS_Start_IAM_Policy_Generation_printer, hdl=hdl, args=args)

'c624cb27-aba6-46a2-b0f2-b4e7b4b8ec87'


In [15]:
##
# Copyright (c) 2023 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field, SecretStr
from typing import Dict, List
import pprint

from beartype import beartype
@beartype
def aws_get_generated_policy_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_get_generated_policy(handle, region:str,jobId:str) -> Dict:
    client = handle.client('accessanalyzer', region_name=region)
    response = client.get_generated_policy(
        jobId=jobId,
        includeResourcePlaceholders=True,
        includeServiceLevelTemplate=True
    )
    return response

task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "doug_admin",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "662e6529-6610-40ae-82be-ed3d57ed1c9b"
}''')
task.configure(inputParamsJson='''{
    "jobId": "jobId",
    "region": "region"
    }''')
task.configure(outputName="generatedPolicy")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_generated_policy, lego_printer=aws_get_generated_policy_printer, hdl=hdl, args=args)

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '1765',
                                      'content-type': 'application/json',
                                      'date': 'Thu, 13 Apr 2023 21:54:18 GMT',
                                      'x-amz-apigw-id': 'DVgBnEYRvHcFzmQ=',
                                      'x-amzn-requestid': 'd5a52032-7dcf-4463-a4fa-66ae56779325',
                                      'x-amzn-trace-id': 'Root=1-64387a0a-70faa16f2917ab9f6af8eccc'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'd5a52032-7dcf-4463-a4fa-66ae56779325',
                      'RetryAttempts': 0},
 'generatedPolicyResult': {'generatedPolicies': [{'policy': '{"Version":"2012-10-17","Statement":[{"Sid":"SupportedServiceSid0","Effect":"Allow","Action":["redshift-data:DescribeStatement","redshift-data:GetStatementResult"],"Resource":"*"},{"Sid":"SupportedServiceSid1","Effect

In [16]:
print(generatedPolicy['jobDetails']['status'])



SUCCEEDED


In [17]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field, SecretStr
from typing import Dict, List
import pprint

from beartype import beartype
@beartype
def aws_get_acount_number_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_get_acount_number(handle) -> str:
    # Create a client object for the AWS Identity and Access Management (IAM) service
    iam_client = handle.client('iam')

    # Call the get_user() method to get information about the current user
    response = iam_client.get_user()

    # Extract the account ID from the ARN (Amazon Resource Name) of the user
    account_id = response['User']['Arn'].split(':')[4]

    # Print the account ID
    return account_id


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "doug_admin",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "662e6529-6610-40ae-82be-ed3d57ed1c9b"
}''')
task.configure(outputName="accountNumber")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_acount_number, lego_printer=aws_get_acount_number_printer, hdl=hdl, args=args)

'100498623390'


In [38]:
import json
import re 


policy = generatedPolicy['generatedPolicyResult']['generatedPolicies'][0]['policy']
#print(policy)

policy = json.dumps(policy)

policy = policy.replace('${Region}', "us-west-2")
policy = policy.replace('${Account}', accountNumber)
policy = re.sub("\${[A-Za-z]*}", "*", policy)
policy = json.loads(policy)
policy = str(policy)
print(type(policy), policy)


<class 'str'> {"Version":"2012-10-17","Statement":[{"Sid":"SupportedServiceSid0","Effect":"Allow","Action":["redshift-data:DescribeStatement","redshift-data:GetStatementResult"],"Resource":"*"},{"Sid":"SupportedServiceSid1","Effect":"Allow","Action":"kms:Decrypt","Resource":"arn:aws:kms:us-west-2:100498623390:key/*"},{"Sid":"SupportedServiceSid2","Effect":"Allow","Action":"redshift-data:ExecuteStatement","Resource":"arn:aws:redshift-serverless:us-west-2:100498623390:workgroup/*"},{"Sid":"SupportedServiceSid3","Effect":"Allow","Action":"redshift-data:ExecuteStatement","Resource":"arn:aws:redshift:us-west-2:100498623390:cluster:*"},{"Sid":"SupportedServiceSid4","Effect":"Allow","Action":"secretsmanager:GetSecretValue","Resource":"arn:aws:secretsmanager:us-west-2:100498623390:secret:*"}]}


In [41]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field, SecretStr
from typing import Dict, List
import pprint

from beartype import beartype
@beartype
def aws_create_IAMpolicy_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_create_IAMpolicy(handle, policyDocument:str, PolicyName:str) -> Dict:

    client = handle.client('iam')
    response = client.create_policy(
        PolicyName=PolicyName,
        PolicyDocument=policyDocument,
        Description='generated Via unSkript',

    )
    return response

task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "doug_admin",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "662e6529-6610-40ae-82be-ed3d57ed1c9b"
}''')
task.configure(inputParamsJson='''{
    "PolicyName": "policy_name",
    "policyDocument": "policy"
    }''')
task.configure(outputName="createdPolicy")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_create_IAMpolicy, lego_printer=aws_create_IAMpolicy_printer, hdl=hdl, args=args)

{'Policy': {'Arn': 'arn:aws:iam::100498623390:policy/generated_iam_policy_11',
            'AttachmentCount': 0,
            'CreateDate': datetime.datetime(2023, 4, 14, 16, 38, 15, tzinfo=tzlocal()),
            'DefaultVersionId': 'v1',
            'IsAttachable': True,
            'Path': '/',
            'PermissionsBoundaryUsageCount': 0,
            'PolicyId': 'ANPAROZRPJOPGEL4G2MMC',
            'PolicyName': 'generated_iam_policy_11',
            'UpdateDate': datetime.datetime(2023, 4, 14, 16, 38, 15, tzinfo=tzlocal())},
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '781',
                                      'content-type': 'text/xml',
                                      'date': 'Fri, 14 Apr 2023 16:38:15 GMT',
                                      'x-amzn-requestid': '6506c9c1-d9c4-4117-ab81-e0139f204de0'},
                      'HTTPStatusCode': 200,
                      'RequestId': '6506c9c1-d9c4-4117-ab81-e0139f204de0',
                      'RetryAttempts

In [18]:
print(createdPolicy['Policy']['Arn'])

arn:aws:iam::100498623390:policy/generated_iam_policy_9


In [42]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from botocore.exceptions import ClientError
import pprint
from beartype import beartype


from beartype import beartype
@beartype
def aws_create_iam_user_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
@beartype
def aws_create_iam_user(handle, user_name: str, tag_key: str, tag_value: str) -> Dict:
    """aws_create_iam_user Creates new IAM User.

        :type handle: object
        :param handle: Object returned by the task.validate(...) method

        :type user_name: string
        :param user_name: Name of new IAM User.

        :type tag_key: string
        :param tag_key: Tag Key assign to new User.

        :type tag_value: string
        :param tag_value: Tag Value assign to new User.

        :rtype: Dict with the stopped instances state info.
    """

    ec2Client = handle.client("iam")
    result = {}
    try:
        response = ec2Client.create_user(
            UserName=user_name,
            Tags=[
                {
                    'Key': tag_key,
                    'Value': tag_value
                }])
        result = response
    except ClientError as error:
        if error.response['Error']['Code'] == 'EntityAlreadyExists':
            result = error.response
        else:
            result = error.response

    return result


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "doug_admin",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "662e6529-6610-40ae-82be-ed3d57ed1c9b"
}''')
task.configure(inputParamsJson='''{
    "tag_key": "\\"test\\"",
    "tag_value": "\\"test\\"",
    "user_name": "user_name"
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_create_iam_user, lego_printer=aws_create_iam_user_printer, hdl=hdl, args=args)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '621',
                                      'content-type': 'text/xml',
                                      'date': 'Fri, 14 Apr 2023 16:38:19 GMT',
                                      'x-amzn-requestid': 'abf342ea-24a0-4021-92b4-8c1e737a76a1'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'abf342ea-24a0-4021-92b4-8c1e737a76a1',
                      'RetryAttempts': 0},
 'User': {'Arn': 'arn:aws:iam::100498623390:user/Doug_generated_iam_14',
          'CreateDate': datetime.datetime(2023, 4, 14, 16, 38, 20, tzinfo=tzlocal()),
          'Path': '/',
          'Tags': [{'Key': 'test', 'Value': 'test'}],
          'UserId': 'AIDAROZRPJOPJJCXE5C6Q',
          'UserName': 'Doug_generated_iam_14'}}


In [22]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from botocore.exceptions import ClientError
import pprint


from beartype import beartype
@beartype
def aws_attach_iam_policy_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_attach_iam_policy(handle, user_name: str, policy_name: str) -> Dict:
    """aws_attache_iam_policy used to provide user permissions.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type user_name: string
        :param user_name: Dictionary of credentials info.

        :type policy_name: string
        :param policy_name: Policy name to apply the permissions to the user.

        :rtype: Dict with User policy information.
    """
    result = {}
    iamResource = handle.resource('iam')
    try:
        user = iamResource.User(user_name)
        response = user.attach_policy(
            PolicyArn='arn:aws:iam::aws:policy/'+policy_name
            )
        result = response
    except ClientError as error:
        result = error.response

    return result


def unskript_default_printer(output):
    if isinstance(output, (list, tuple)):
        for item in output:
            print(f'item: {item}')
    elif isinstance(output, dict):
        for item in output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(output)

task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "doug_admin",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "ccdef705-2e5a-4bc5-9709-fbb8eb1057df"
}''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_attach_iam_policy, lego_printer=unskript_default_printer, hdl=hdl, args=args)

Error: Parameter type validation failed: aws_attach_iam_policy() missing 2 required positional arguments: 'user_name' and 'policy_name'


In [43]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field, SecretStr
from typing import Dict, List
import pprint

from beartype import beartype
@beartype
def aws_list_all_iam_users_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_list_all_iam_users(handle, user:str, policy:str) -> Dict:
    # Create an IAM client
    iam = handle.client('iam')
    print(policy)
    # Define the IAM user name and policy arn
    user_name = user
    policy_arn = policy

    # Attach the policy to the user
    response = iam.attach_user_policy(
        UserName=user_name,
        PolicyArn=policy_arn
    )

    return response

task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "doug_admin",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "662e6529-6610-40ae-82be-ed3d57ed1c9b"
}''')
task.configure(inputParamsJson='''{
    "policy": "createdPolicy['Policy']['Arn']",
    "user": "user_name"
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_all_iam_users, lego_printer=aws_list_all_iam_users_printer, hdl=hdl, args=args)

arn:aws:iam::100498623390:policy/generated_iam_policy_11
{'ResponseMetadata': {'HTTPHeaders': {'content-length': '212',
                                      'content-type': 'text/xml',
                                      'date': 'Fri, 14 Apr 2023 16:38:29 GMT',
                                      'x-amzn-requestid': 'e8a1bdad-78db-4495-b784-1d9e9145ad21'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'e8a1bdad-78db-4495-b784-1d9e9145ad21',
                      'RetryAttempts': 0}}
